In [ ]:
train = read.csv(file = '../input/457-engineered/train_engineered (2).csv', header = TRUE)
test = read.csv(file = '../input/457-engineered/test_engineered (2).csv', header = TRUE)
train_sub = train[train$Sales > 0, ]#only use positive Sales in training data
testOpen = (test$Open == 1)
test_sub = test[testOpen, ] #only needs to do prediction for open days
colnames(train_sub)

from feature selection, our current models and feature selected are:

Customer:

lm(formula = Customers ~ factor(Store) + factor(DayOfWeek) + 
    Promo + SchoolHoliday + factor(year) + factor(month) + day + 
    factor(Store):factor(DayOfWeek) + factor(Store):Promo + factor(Store):SchoolHoliday + 
    factor(Store):factor(year) + factor(Store):factor(month) + 
    factor(Store):day + factor(DayOfWeek):SchoolHoliday + 
    factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    factor(DayOfWeek):day + Promo:factor(year) + Promo:factor(month) + 
    Promo:day + SchoolHoliday:factor(month) + SchoolHoliday:day + factor(month):day + CompetitionDistance + StoreType, data = train_sub)


Sales:

lm(formula = Sales ~ factor(Store) + factor(DayOfWeek) + Customers + 
    Promo + SchoolHoliday + factor(year) + factor(month) + day + 
    factor(Store):factor(DayOfWeek) + factor(Store):Customers + 
    factor(Store):Promo + factor(Store):SchoolHoliday + factor(Store):factor(year) + 
    factor(Store):factor(month) + factor(Store):day + factor(DayOfWeek):Customers + 
    factor(DayOfWeek):Promo + factor(DayOfWeek):SchoolHoliday + 
    factor(DayOfWeek):factor(year) + factor(DayOfWeek):factor(month) + 
    Customers:Promo + Customers:SchoolHoliday + Customers:factor(year) + 
    Customers:factor(month) + Customers:day + Promo:SchoolHoliday + 
    Promo:factor(year) + Promo:factor(month) + Promo:day + SchoolHoliday:factor(year) + 
    SchoolHoliday:factor(month) + factor(year):factor(month) + 
    factor(year):day + factor(month):day + CompetitionDistance, data = train_sub)

In [ ]:
library(randomForest)
set.seed(123)
trainID = sample(1:nrow(train_sub), round(nrow(train_sub)*3/4))
rfModel = randomForest(Sales ~ ., 
                       data = train_sub, 
                       subset = trainID,
                       importance = TRUE, 
                       ntree=1000)

In [ ]:
plot(rfModel)

In [ ]:
library(MLmetrics)
rf.pred = predict(rfModel, train_sub[-trainID, ])
RMSPE(y_pred = rf.pred, y_true = train_sub[-trainID,3])

In [ ]:
library(caret)
ctrl <- trainControl(method = "cv",
                     number = 10)

rf.Grid = expand.grid(mtry = 2:9) 
rf.cv.model = train(Sales ~ ., data = train_sub[trainID, ],
                  method = "rf",
                  trControl = ctrl,
                  tuneGrid = rf.Grid)


In [ ]:
rf.cv.model

In [ ]:
rf.cv.model$bestTune

try Regularized Random Forest(RRF, parameter mtry, coefReg, coefImp)

Considering to use dummy variable, but it inducing sparsity into the dataset which is undesirable.

In [ ]:
train_sub_factors <- train_sub
for(i in c(1,2,8,9,10)){
    train_sub_factors[,i] <- factor(train_sub_factors[,i])
}



In [ ]:
train_sub_dummies <- fastDummies::dummy_cols(train_sub_factors, remove_first_dummy = TRUE)
train_sub_dummies

# Customers

In [ ]:
trainID = sample(1:nrow(train_sub), round(nrow(train_sub)*3/4))
rfModel = randomForest(Customers ~ ., 
                       data = train_sub[,-3], 
                       subset = trainID,
                       importance = TRUE, 
                       ntree=1000,
                        mtry = 6)

In [ ]:
library(MLmetrics)
rf.pred = predict(rfModel, train_sub[-trainID,-3])
RMSPE(y_pred = rf.pred, y_true = train_sub[-trainID,4])

0.0793932200112622, 0.118335807600756